In [159]:
# %%writefile Factor_Cacher.py

import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import warnings
import pandas as pd
import numpy as np

import tools.Sample_Tools as smpl
import base.JuUnits as ju


__cache_dir__ = ju.get_root_path()+'/cache.feather/'
__cache_type__ = '.feather'

# df = smpl.get_sample_by_zs(name='沪深300', end='2021-11-28', gap=2500,  only_main=True, filter_st=True).data
# df.to_parquet(__cache_dir__+'test.parquet')

def file_path(file_name):
    return __cache_dir__ + file_name + __cache_type__

def is_cache_exist(cache_name):
    return os.path.exists(file_path(cache_name))

def save_cache(name, data):
    data.reset_index().to_feather(file_path(name))
    
def load_cache(cache_name:str, to_series:bool=False):
    assert is_cache_exist(cache_name),'cache not exist, create first'
    df = pd.read_feather(file_path(cache_name)).set_index(['date','code']).sort_index()
    if to_series:
        df =  df.squeeze()
    return df

def load_cache_adv(cache_name:str, start:str, end:str, to_series:bool=False, flag_time:str='date'):
    assert is_cache_exist(cache_name),'cache not exist, create first'
        
    st = pd.Timestamp(start) 
    en = pd.Timestamp(end) 

    df = pd.read_feather(file_path(cache_name)).set_index([flag_time,'code']).sort_index()[st:en]
    date_idx = df.index.get_level_values(0).unique()
    if not (st >= date_idx.min() and en <= date_idx.max()):
        info = '"{}"：date range out of cache[{},{}]'.format(
            cache_name,
            date_idx.min().strftime('%Y-%m-%d %H:%M:%S'), 
            date_idx.max().strftime('%Y-%m-%d %H:%M:%S')
        )
        warnings.warn(info)
        
    df = df[st:en]
    
    if to_series:
        df =  df.squeeze()
    return df


def load_caches_adv(cache_names:list, start:str=None, end:str=None, flag_time:str='date'):
    if not start is None:
        assert not end is None, 'start and end must not be None at the same time,or be None at the same time'
    
    tmp = []
    for name in cache_names:
        if start is None:
            df = load_cache(name)
        else:
            df = load_cache_adv(name,start,end,flag_time=flag_time)
        tmp.append(df)  
        
    return pd.concat(tmp,axis=1)
     
    
        

In [31]:
%%time
df.reset_index().to_feather(__cache_dir__+'test.feather')

Wall time: 71.8 ms


In [162]:
# df['open'].reset_index().to_feather(__cache_dir__+'test_s.feather')
# pd.read_feather(__cache_dir__+'test_s.feather').set_index(['date','code']).squeeze()
# df.squeeze()
# load_cache_adv('test','2011-11-11','2022-11-11')
load_caches_adv(['alpha11','alpha24'],'2011-11-11','2020-11-11')

alpha11  alpha24
date       code                     
2011-11-11 000001  0.468002     0.17
           000002  0.134449     0.16
           000063  0.301422     0.19
           000066  1.043020    -0.44
           000069  0.399679     0.15
...                     ...      ...
2020-11-11 603882  1.653110    15.08
           603899  0.482447    -2.20
           603939  1.360609     1.28
           603986  1.024721   -28.90
           603993  0.146505     0.04

[449688 rows x 2 columns]